In [1]:
import pandas as pd
import seaborn as sns
from dataprep.eda import *
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['font.family'] = 'AppleGothic'

NumExpr defaulting to 8 threads.


base line 불러오기

In [3]:
df=pd.read_excel('/Users/bomi/bc/base_line.xlsx')

In [4]:
df.head()

,broadcast,exp_min,m_code,p_code,p_name,p_group,unit_price,total_price,sales,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,YEAR_DAY,DAY_NAME,DAY_NUM,code,no
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000,53,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000,110,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,1
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000,82,2019,1,1,6,20,0,2019-01-01,Tuesday,2,0,2
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000,174,2019,1,1,6,20,0,2019-01-01,Tuesday,2,0,3
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000,167,2019,1,1,6,40,0,2019-01-01,Tuesday,2,0,4


추가 데이터 불러오기 :
기상청01_지상(종관,ASOS)시간자료_조회서비스_오픈API from 공공데이터포털

In [97]:
import requests
from bs4 import BeautifulSoup

In [226]:
tm_list=[] #시간
ta_list=[] #기온
rn_list=[] #강수량
hm_list=[] #습도
ss_list=[] #일조

for i in range(1,19):
    key='JIhQPZFK66iLNy5Sft4JFuU3LvLtZByKmwbu2ac5h7jFG86WGaV6uhmfHoXhQz%2FHl3Ttr7NlYA%2FxlKT1IyHdKg%3D%3D'
    url='http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey={}'.format(key)
    params={'numOfRows':'500',
            'pageNo': i,
            'dataCd':'ASOS',
            'dataType':'JSON',
            'dateCd':'HR',
            'stnIds':'108',
            'endDt':'20200101',
            'endHh':'02',
            'startHh':'06',
            'startDt':'20190101'}
    resp=requests.get(url,params=params).content
    json_object= json.loads(resp)
    json_item=json_object['response']['body']['items']['item']

    for i in range(len(json_item)):
        tm=json_item[i]['tm']
        tm_list.append(tm)

        ta=json_item[i]['ta']
        ta_list.append(ta)

        rn=json_item[i]['rn']
        rn_list.append(rn)

        hm=json_item[i]['hm']
        hm_list.append(hm)

        ss=json_item[i]['ss']
        ss_list.append(ss)

In [228]:
tm_list[-1]

'2020-01-01 02:00'

In [229]:
df1=pd.DataFrame(data={'time':tm_list,
                      'temp':ta_list,
                      'rain':rn_list,
                      'humidity':hm_list,
                      'sunshine':ss_list})

In [230]:
df1

,time,temp,rain,humidity,sunshine
0,2019-01-01 06:00,-7.9,0.0,60,0.0
1,2019-01-01 07:00,-7.7,0.0,58,0.0
2,2019-01-01 08:00,-7.7,0.0,57,0.0
3,2019-01-01 09:00,-7.0,0.0,50,0.0
4,2019-01-01 10:00,-4.9,0.0,44,0.6
...,...,...,...,...,...
8752,2019-12-31 22:00,-8.2,0.0,42,0.0
8753,2019-12-31 23:00,-7.4,0.0,45,0.0
8754,2020-01-01 00:00,-6.5,0.0,38,0.0
8755,2020-01-01 01:00,-5.9,0.0,40,0.0


In [236]:
from datetime import datetime

In [241]:
df1['datetime']=df1['time'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M'))

In [246]:
del df1['time']

In [248]:
df1

,temp,rain,humidity,sunshine,datetime
0,-7.9,0.0,60,0.0,2019-01-01 06:00:00
1,-7.7,0.0,58,0.0,2019-01-01 07:00:00
2,-7.7,0.0,57,0.0,2019-01-01 08:00:00
3,-7.0,0.0,50,0.0,2019-01-01 09:00:00
4,-4.9,0.0,44,0.6,2019-01-01 10:00:00
...,...,...,...,...,...
8752,-8.2,0.0,42,0.0,2019-12-31 22:00:00
8753,-7.4,0.0,45,0.0,2019-12-31 23:00:00
8754,-6.5,0.0,38,0.0,2020-01-01 00:00:00
8755,-5.9,0.0,40,0.0,2020-01-01 01:00:00


In [250]:
df1.to_csv('weather_time_seoul.csv')

In [251]:
df1

,temp,rain,humidity,sunshine,datetime
0,-7.9,0.0,60,0.0,2019-01-01 06:00:00
1,-7.7,0.0,58,0.0,2019-01-01 07:00:00
2,-7.7,0.0,57,0.0,2019-01-01 08:00:00
3,-7.0,0.0,50,0.0,2019-01-01 09:00:00
4,-4.9,0.0,44,0.6,2019-01-01 10:00:00
...,...,...,...,...,...
8752,-8.2,0.0,42,0.0,2019-12-31 22:00:00
8753,-7.4,0.0,45,0.0,2019-12-31 23:00:00
8754,-6.5,0.0,38,0.0,2020-01-01 00:00:00
8755,-5.9,0.0,40,0.0,2020-01-01 01:00:00
